# Сбор данных

Соберите отзывы (3-5 разных классов) о Россельхозбанке с сайта Банки.ру. Например, класс “Обслуживание” или “Кредит”.

Можно воспользоваться [ссылкой на сайт](https://www.banki.ru/services/responses/bank/rshb/).

Способ получения данных выберите самостоятельно — можете собирать вручную или написать код для парсинга данных.

Убедитесь, что вы имеете достаточно данных по каждому классу.
Чем больше отзывов вы соберете — тем лучше.

Выберу классы: <br>
1. Дебетовая карта *(83 страницы)*
2. Ипотека *(117 страниц)*
3. Потребительский кредит *(53 страницы)*
4. Вклад *(74 страницы)*

Будут собраны следующие данные для каждого класса:
1. Заголовок
2. Текст отзыва
3. URL отзыва
4. Оценка
5. Дата
6. Класс {'debitcard', 'credit', 'deposit', 'hypothec'}

Для парсинга сайта будет использоваться библиотека Beautiful Soup, для запросов - requests.

In [2]:
# импортируем необходимые библиотеки
import requests 
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [3]:
# Ссылки: 
# Потребительский кредит:
link_credits = 'https://www.banki.ru/services/responses/bank/rshb/product/credits/?page='
# Ипотека: 
link_hypothec = 'https://www.banki.ru/services/responses/bank/rshb/product/hypothec/?page='
# Дебетовая карта: 
link_debitcards = 'https://www.banki.ru/services/responses/bank/rshb/product/debitcards/?page='
# Вклад: 
link_deposits = 'https://www.banki.ru/services/responses/bank/rshb/product/deposits/?page='

In [4]:
# получить список словарей данных для страницы (уже прочитанной bs) и присвоить им класс label 
def get_info_for_bs_page(bs, label):
    responses = bs.find_all(class_='responses__item')
    result = [{
            'title': i.find('a').text,
            'url': 'banki.ru' + i.find('a')['href'],
            'text': i.find(class_='responses__item__message markup-inside-small markup-inside-small--bullet').text.strip(), 
            'score': int(i.find(class_='rating-grade').text.strip()) if i.find(class_='rating-grade') else None,
            'date': i.find('time', class_='display-inline-block').text,
            'label': label
    } for i in responses]
        
    return result

In [5]:
# получить все страницы по выбранной ссылке (link_reviews) с выбранным классом (label)
def get_all_reviews(link_reviews, label):
    # изменим заголовки, чтобы сайт не блокировал запросы
    session = requests.session() 
    session.headers['Accept'] = \
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    session.headers['User-Agent'] = \
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'
    # чтобы сайт не посчитал нас Ddos
    session.keep_alive = False

    all_reviews = []
    i = 1
    while 1:
        try:
            req = session.get(link_reviews + str(i))
        except:
            print('Произошла ошибка подключения, повторнре подключение')
            req = session.get(link_reviews + str(i))
        else: 
            if req.status_code == 404:
                break
            bs = BeautifulSoup(req.text)
            all_reviews.extend(get_info_for_bs_page(bs, label))
            print(f'Загружены данные {label}, для страницы {i}')
            i += 1
    return all_reviews

### Кредит

In [6]:
all_credit = get_all_reviews(link_credits, 'credit')

Got results for credit, page 1
Got results for credit, page 2
Got results for credit, page 3
Got results for credit, page 4
Got results for credit, page 5
Got results for credit, page 6
Got results for credit, page 7
Got results for credit, page 8
Got results for credit, page 9
Got results for credit, page 10
Got results for credit, page 11
Got results for credit, page 12
Got results for credit, page 13
Got results for credit, page 14
Got results for credit, page 15
Got results for credit, page 16
Got results for credit, page 17
Got results for credit, page 18
Got results for credit, page 19
Got results for credit, page 20
Got results for credit, page 21
Got results for credit, page 22
Got results for credit, page 23
Got results for credit, page 24
Got results for credit, page 25
Got results for credit, page 26
Got results for credit, page 27
Got results for credit, page 28
Got results for credit, page 29
Got results for credit, page 30
Got results for credit, page 31
Got results for c

In [8]:
all_credit_df = pd.DataFrame(all_credit)

In [9]:
all_credit_df.head()

,title,url,text,score,date,label
0,благодарность,banki.ru/services/responses/bank/response/1044...,ДОБРЫЙ ДЕНЬ !!! ХОЧУ написать БЛАГОДАРНОСТЬ...,5.0,03.12.2020 13:53,credit
1,Числится задолженность по закрытым кредитам в ...,banki.ru/services/responses/bank/response/1044...,добрый день! подавал заявки на кредит в банки ...,NaN,02.12.2020 11:18,credit
2,Досрочное погашение кредита,banki.ru/services/responses/bank/response/1044...,Требование решить вопрос о корректном закрытии...,1.0,01.12.2020 0:13,credit
3,"Жуткий колхоз, незаконно повышают % по кредиту",banki.ru/services/responses/bank/response/1044...,Осенью 2019 г. взял потребительский кредит.25....,1.0,30.11.2020 21:52,credit
4,Хорошая работа,banki.ru/services/responses/bank/response/1044...,"Здравствуйте, хотел бы оставить отзыв о хороше...",5.0,30.11.2020 15:31,credit


In [10]:
all_credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   1315 non-null   object 
 1   url     1315 non-null   object 
 2   text    1315 non-null   object 
 3   score   936 non-null    float64
 4   date    1315 non-null   object 
 5   label   1315 non-null   object 
dtypes: float64(1), object(5)
memory usage: 61.8+ KB


### Ипотека

In [11]:
all_hypothec = get_all_reviews(link_hypothec, 'hypothec')

Got results for hypothec, page 1
Got results for hypothec, page 2
Got results for hypothec, page 3
Got results for hypothec, page 4
Got results for hypothec, page 5
Got results for hypothec, page 6
Got results for hypothec, page 7
Got results for hypothec, page 8
Got results for hypothec, page 9
Got results for hypothec, page 10
Got results for hypothec, page 11
Got results for hypothec, page 12
Got results for hypothec, page 13
Got results for hypothec, page 14
Got results for hypothec, page 15
Got results for hypothec, page 16
Got results for hypothec, page 17
Got results for hypothec, page 18
Got results for hypothec, page 19
Got results for hypothec, page 20
Got results for hypothec, page 21
Got results for hypothec, page 22
Got results for hypothec, page 23
Got results for hypothec, page 24
Got results for hypothec, page 25
Got results for hypothec, page 26
Got results for hypothec, page 27
Got results for hypothec, page 28
Got results for hypothec, page 29
Got results for hypothe

In [13]:
all_hypothec_df = pd.DataFrame(all_hypothec)

In [14]:
all_hypothec_df.head()

,title,url,text,score,date,label
0,Благодарность сотруднику,banki.ru/services/responses/bank/response/1044...,"Добрый день!👋 Я являюсь клиентом вашего банка,...",5.0,04.12.2020 21:21,hypothec
1,Доступная ипотека,banki.ru/services/responses/bank/response/1044...,В середине сентябре 2020 я подала заявку на ип...,1.0,04.12.2020 18:16,hypothec
2,Отвратительное обслуживание,banki.ru/services/responses/bank/response/1044...,"Обратились в этот банк еще в сентябре, подали ...",1.0,04.12.2020 17:27,hypothec
3,Абсолютно не заинтересованный в клиентах офис,banki.ru/services/responses/bank/response/1044...,"Обратились за ипотечным кредитом, но персонал ...",1.0,04.12.2020 15:55,hypothec
4,Отзыв.,banki.ru/services/responses/bank/response/1044...,Добрый день ! Я в возрасте 50 лет осуществила ...,4.0,04.12.2020 15:46,hypothec


### Дебетовые карты

In [16]:
all_debitcards = get_all_reviews(link_debitcards, 'debitcard')

Got results for debitcard, page 1
Got results for debitcard, page 2
Got results for debitcard, page 3
Got results for debitcard, page 4
Got results for debitcard, page 5
Got results for debitcard, page 6
Got results for debitcard, page 7
Got results for debitcard, page 8
Got results for debitcard, page 9
Got results for debitcard, page 10
Got results for debitcard, page 11
Got results for debitcard, page 12
Got results for debitcard, page 13
Got results for debitcard, page 14
Got results for debitcard, page 15
Got results for debitcard, page 16
Got results for debitcard, page 17
Got results for debitcard, page 18
Got results for debitcard, page 19
Got results for debitcard, page 20
Got results for debitcard, page 21
Got results for debitcard, page 22
Got results for debitcard, page 23
Got results for debitcard, page 24
Got results for debitcard, page 25
Got results for debitcard, page 26
Got results for debitcard, page 27
Got results for debitcard, page 28
Got results for debitcard, pa

In [18]:
all_debitcards_df = pd.DataFrame(all_debitcards)

In [19]:
all_debitcards_df.head()

,title,url,text,score,date,label
0,Ужасное обслуживание зарплатного проекта,banki.ru/services/responses/bank/response/1044...,Я-госслужащий. Зарплатные проект обслуживается...,1.0,04.12.2020 22:43,debitcard
1,Не хотят закрывать карты,banki.ru/services/responses/bank/response/1044...,Первое обращение.30.09.2020 г. по моей просьб...,1.0,04.12.2020 20:14,debitcard
2,"Мутные балы программы лояльности ""Урожай""",banki.ru/services/responses/bank/response/1044...,В банках и магазинах уже давно прописаны довол...,1.0,04.12.2020 13:37,debitcard
3,"Никто ничего не делает! Клиенты? Не, не слышали.",banki.ru/services/responses/bank/response/1044...,25 сентября 2019 года закрыла карту Амурский т...,1.0,04.12.2020 11:04,debitcard
4,Самый худший банк города Иваново,banki.ru/services/responses/bank/response/1044...,"В сентябре месяце муж обратился в банк , нужны...",1.0,04.12.2020 10:04,debitcard


In [20]:
all_debitcards_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2046 entries, 0 to 2045
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   2046 non-null   object 
 1   url     2046 non-null   object 
 2   text    2046 non-null   object 
 3   score   1694 non-null   float64
 4   date    2046 non-null   object 
 5   label   2046 non-null   object 
dtypes: float64(1), object(5)
memory usage: 96.0+ KB


### Вклад

In [21]:
all_deposits = get_all_reviews(link_deposits, 'deposit')

Got results for deposit, page 1
Got results for deposit, page 2
Got results for deposit, page 3
Got results for deposit, page 4
Got results for deposit, page 5
Got results for deposit, page 6
Got results for deposit, page 7
Got results for deposit, page 8
Got results for deposit, page 9
Got results for deposit, page 10
Got results for deposit, page 11
Got results for deposit, page 12
Got results for deposit, page 13
Got results for deposit, page 14
Got results for deposit, page 15
Got results for deposit, page 16
Got results for deposit, page 17
Got results for deposit, page 18
Got results for deposit, page 19
Got results for deposit, page 20
Got results for deposit, page 21
Got results for deposit, page 22
Got results for deposit, page 23
Got results for deposit, page 24
Got results for deposit, page 25
Got results for deposit, page 26
Got results for deposit, page 27
Got results for deposit, page 28
Got results for deposit, page 29
Got results for deposit, page 30
Got results for dep

In [23]:
all_deposits_df = pd.DataFrame(all_deposits)

In [24]:
all_deposits_df.head()

,title,url,text,score,date,label
0,Не возвращают вклад,banki.ru/services/responses/bank/response/1044...,У меня открыт вклад в отделении г. Череповец В...,1.0,03.12.2020 23:12,deposit
1,Благодарность сотрудникам отделения и обращени...,banki.ru/services/responses/bank/response/1044...,Добрый день ! Я обращаюсь к руководству Москов...,5.0,03.12.2020 15:16,deposit
2,Незаконно присвоили 600 рублей!,banki.ru/services/responses/bank/response/1044...,Украли 600 рублей! Три года назад открыл у них...,1.0,03.12.2020 11:35,deposit
3,Отвратительное обслуживание,banki.ru/services/responses/bank/response/1044...,Отвратительное обслуживание. Дозвониться до ну...,1.0,02.12.2020 19:28,deposit
4,Достали своим SMS-спамом,banki.ru/services/responses/bank/response/1044...,Когда-то давно был вклад в РСХБ. Последние нес...,1.0,02.12.2020 13:15,deposit


In [25]:
all_deposits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1831 entries, 0 to 1830
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   1831 non-null   object 
 1   url     1831 non-null   object 
 2   text    1831 non-null   object 
 3   score   1449 non-null   float64
 4   date    1831 non-null   object 
 5   label   1831 non-null   object 
dtypes: float64(1), object(5)
memory usage: 86.0+ KB


### Создание единого DataFrame и сохранение в csv

In [29]:
reviews_df = pd.concat([all_credit_df, all_hypothec_df, all_debitcards_df, all_deposits_df])

In [32]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8107 entries, 0 to 1830
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   8107 non-null   object 
 1   url     8107 non-null   object 
 2   text    8107 non-null   object 
 3   score   6354 non-null   float64
 4   date    8107 non-null   object 
 5   label   8107 non-null   object 
dtypes: float64(1), object(5)
memory usage: 443.4+ KB


In [33]:
reviews_df.to_csv('rshb_reviews.csv', index=False)

Таким образом, были получены все отзывы по четырем классам услуг с сайта banki.ru. При необходимости можно расширить выборку, просто добавив название нового файла. 